# Isoscape dataset extraction

Author: Martina Kauzlaric (martina.kauzlaric@unibe.ch)

This notebook is used to extract data from monthly isoscapes (i.e. isotopic landscapes, which are spatially continuous and georeferenced isotope datasets) of deuterium [‰] and oxygen-18 [‰] with a resolution of 500m provided by the FOEN into a table for publication alongisde the used data.
Stable isotope data in precipitation of the isotope observation network in Switzerland (ISOT), together with several influencing variables (e.g., topographical parameters, climate variables) are used in a multi-regression framework, and the residuals are interpolated by the use of ordinary kriging.
Monthly data are available for the years between 2007 and 2023.

## Requirements
**Python:**

* Python=3.13.2
* Jupyter
* os
* numpy=2.2.4
* xarray=2024.11.0
* pandas=2.2.3
* geopandas=1.0.1
* cartopy=0.24.1
* matplotlib=3.10.0
* tqdm=4.67.1

Check the Github repository for an environment_landcover.yml (for conda environments) file [for semplicity we use the same environment and that used for extracting the landcover data]

**Files:**

Monthly rasters, e.g.
* iso_sum_ras_simple_GEO+N_2H_20070115.asc
* iso_sum_ras_simple_GEO+N_18O_20070115.asc


**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References
* Bundesamt für Umwelt BAFU
* Office fédéral de l'environnement OFEV
* Ufficio federale dell'ambiente UFAM
* Federal Office for the Environment FOEN
* © BAFU
* https://www.bafu.admin.ch/bafu/en/home/topics/water/groundwater/groundwater-resources/stable-water-isotopes.html
* https://www.bafu.admin.ch/dam/bafu/de/dokumente/hydrologie/externe-studien-berichte/isoscapes_schweiz_endbericht.pdf.download.pdf/isoscapes_schweiz_endbericht.pdf
## Observations
* Data are only available for catchments inside the national boundaries!

# Import modules

In [2]:
# Clear all variables
%reset -f
#Import necessary libraries
import os
import glob
import numpy as np
#import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon
from shapely.geometry import box
import tqdm as tqdm
import re
import rasterio
from rasterio.mask import mask
from concurrent.futures import ThreadPoolExecutor # this is to run functions in parallel

# Configurations

In [ ]:
# Only editable variables:
# Set (relative) path to your local directory
#PATH = "../.."
PATH = "S:\\CAMELS-CH\\CAMELS-chem"

In [3]:
## Set directories
GIS_dir = os.path.join(PATH,"data\\GIS")
# Define shapefile with the catchments
catchments_shp = os.path.join(GIS_dir,"shapefile_catchments\\camels_ch_chem_catchment_boundaries.shp")
#Add subfolder to GIS_dir for Isoscape data
GIS_dir = os.path.join(GIS_dir, "Isoscapes")  
PATH_OUTPUT = os.path.join(PATH,"results\\catchment_aggregated_data\\isoscapes")

# Create the output directory if it does not exist
if not os.path.isdir(PATH_OUTPUT):
    os.makedirs(PATH_OUTPUT, exist_ok=True)

##Change to directory to where you want to store the results    
os.chdir(PATH_OUTPUT)

In [4]:
os.getcwd()

'c:\\Users\\nascimth\\OneDrive - Eawag\\Eawag\\Papers\\CAMELS_CH_Chem\\GitHub\\camels-ch-chem-paper\\code\\dataset'

* #### The users should NOT change anything in the code below here. 

# Import data
* Load catchments and look at full table

*Note: data are in LV95/CH1903+, i.e. EPSG 2056*

In [5]:
catchments = gpd.read_file(catchments_shp)
catchments["bafu_id"] = catchments["gauge_id"]
catchments

,gauge_id,sensor_id,nawaf_id,nawat_id,isot_id,chirp_id,gauge_name,water_body,gauge_east,gauge_nort,gauge_lon,gauge_lat,area,area_swiss,geometry,bafu_id
0,2009,2009.0,1837.0,1837.0,NIO04,NaN,Porte du Scex,Rhône,557660,133280,6.89,46.35,5239.4,99.994914,"POLYGON Z ((2674253.038 1167429.881 0, 2674340...",2009
1,2011,2011.0,NaN,4070.0,None,NaN,Sion,Rhône,593770,118630,7.36,46.22,3372.4,100.000000,"POLYGON Z ((2674253.038 1167429.881 0, 2674340...",2011
2,2016,2016.0,1833.0,1833.0,NIO02,NaN,Brugg,Aare,657000,259360,8.19,47.48,11681.3,100.000000,"POLYGON Z ((2655969.68 1259695.589 0, 2655976....",2016
3,2018,2018.0,1835.0,1339.0,None,NaN,Mellingen,Reuss,662830,252580,8.27,47.42,3385.8,100.000000,"POLYGON Z ((2663723.38 1252919.068 0, 2663794....",2018
4,2019,2019.0,NaN,1852.0,NIO01,NaN,Brienzwiler,Aare,649930,177380,8.09,46.75,555.2,100.000000,"POLYGON Z ((2669196.412 1183579.51 0, 2669203....",2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,2617,2617.0,NaN,NaN,None,NaN,Müstair,Rom,830800,168700,10.45,46.63,128.6,42.552175,"POLYGON Z ((2820942.826 1171469.984 0, 2820953...",2617
111,2623,2623.0,NaN,NaN,None,NaN,Oberwald,Rhone,669900,154075,8.35,46.53,93.3,100.000000,"POLYGON Z ((2674253.038 1167429.881 0, 2674340...",2623
112,2634,2634.0,6169.0,1181.0,None,NaN,Emmen,Kleine Emme,663700,213630,8.28,47.07,478.3,100.000000,"POLYGON Z ((2653429.237 1216261.807 0, 2653439...",2634
113,2635,2635.0,NaN,NaN,None,NaN,"Einsiedeln, Gross",Grossbach,700710,218125,8.77,47.11,8.9,100.000000,"POLYGON Z ((2701144.527 1218073.633 0, 2701261...",2635


Now we get the isoscapes and extract these per catchment.

*Note: data are in LV95/CH1903+, i.e. EPSG 2056*

In [6]:
# Detect directories containing monthly isoscape data
ascii_dirs = [d for d in os.listdir(GIS_dir) if "Monatsraster" in d]
print(ascii_dirs)

['18O_Monatsraster_2007_2023', '2H_Monatsraster_2007_2023']


In [7]:
# Define isotopes to be extracted
isotopes = ["18O", "2H"]

In [9]:
# --- HELPER FUNCTION FOR ONE CATCHMENT ---
def process_catchment(args):
    catch_idx, catch, raster_files, isotope, crs, isotope_outdir = args

    catch_id = catch["gauge_id"]
    catch_geom = gpd.GeoDataFrame([catch], crs=crs)

    values = []
    months = []

    for file_path in raster_files:
        month_match = re.search(r"_(\d{8})\.asc$", file_path)
        if not month_match:
            continue
        date_str = month_match.group(1)
        date = pd.to_datetime(date_str, format="%Y%m%d")
        months.append(date)

        try:
            with rasterio.open(file_path) as src:
                out_image, _ = mask(src, catch_geom.geometry, crop=True)
                out_image = out_image[0]
                valid = out_image != src.nodata
                if np.any(valid):
                    mean_value = float(np.nanmean(out_image[valid]))
                else:
                    mean_value = np.nan
        except Exception as e:
            print(f"⚠️ Error in file {file_path} for {catch_id}: {e}")
            mean_value = np.nan

        values.append(mean_value)

    # Save timeseries
    df = pd.DataFrame({
        "date": months,
        f"{isotope}_monthlymean": values
    })
    df.to_csv(os.path.join(isotope_outdir, f"{catch_id}_isoscape.csv"),
              index=False, sep=";", float_format="%.2f")
    return catch_id

In [16]:
# --- MAIN LOOP ---
if __name__ == "__main__":
    for isotope in isotopes:
        print(f"\n🧪 Processing isotope {isotope}...")

        dir_match = [d for d in ascii_dirs if isotope in d]
        if not dir_match:
            print(f"⚠️ No directory found for isotope {isotope}")
            continue

        ascii_dir = os.path.join(GIS_dir, dir_match[0])
        raster_files = sorted([
            os.path.join(ascii_dir, f) for f in os.listdir(ascii_dir) if f.endswith(".asc")
        ])

        isotope_outdir = os.path.join(PATH_OUTPUT, isotope)
        os.makedirs(isotope_outdir, exist_ok=True)

        task_args = [
            (idx, row, raster_files, isotope, catchments.crs, isotope_outdir)
            for idx, row in catchments.iterrows()
        ]

        print(f"🚀 Launching parallel processing on {min(20, os.cpu_count())} cores...")

        with ThreadPoolExecutor(max_workers=20) as executor:
            results = list(tqdm.tqdm(executor.map(process_catchment, task_args),
                                     total=len(task_args), desc=f"Catchments for {isotope}"))

        print(f"Done with isotope {isotope} — {len(results)} catchments processed.")


🧪 Processing isotope 18O...
🚀 Launching parallel processing on 20 cores...


Catchments for 18O:   0%|          | 0/115 [00:00<?, ?it/s]p:\Hydrologie\AnacondaMartina\envs\env_CAMELS_CH-chem\Lib\site-packages\rasterio\features.py:392: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _rasterize(valid_shapes, out, transform, all_touched, merge_alg)
p:\Hydrologie\AnacondaMartina\envs\env_CAMELS_CH-chem\Lib\site-packages\rasterio\features.py:392: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _rasterize(valid_shapes, out, transform, all_touched, merge_alg)
Catchments for 18O: 100%|██████████| 115/115 [13:18<00:00,  6.94s/it] 


Done with isotope 18O — 115 catchments processed.

🧪 Processing isotope 2H...
🚀 Launching parallel processing on 20 cores...


Catchments for 2H:  45%|████▌     | 52/115 [08:54<04:43,  4.50s/it]  p:\Hydrologie\AnacondaMartina\envs\env_CAMELS_CH-chem\Lib\site-packages\rasterio\features.py:392: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  _rasterize(valid_shapes, out, transform, all_touched, merge_alg)
Catchments for 2H: 100%|██████████| 115/115 [13:25<00:00,  7.00s/it]

Done with isotope 2H — 115 catchments processed.


Here  following is the code to loop over one catchment per time [depending on the number of catchments and the number of cores available maybe necessary..]

In [ ]:
# --- Helper Function ---
def extract_mean_per_catchment(raster_fp, catchment_geom):
    """Returns the mean of raster values within a catchment geometry."""
    with rasterio.open(raster_fp) as src:
        out_image, out_transform = mask(src, catchment_geom.geometry, crop=True)
        out_image = out_image[0]  # First band

        # Masked values are set to src.nodata
        valid = out_image != src.nodata
        if np.any(valid):
            return float(np.nanmean(out_image[valid]))
        else:
            return np.nan

In [11]:
for isotope in isotopes:
    print(f"\n🧪 Processing isotope {isotope}...")

    # Find the correct directory for this isotope
    dir_match = [d for d in ascii_dirs if isotope in d]
    if not dir_match:
        print(f"⚠️ No directory found for isotope {isotope}")
        continue

    ascii_dir = os.path.join(GIS_dir, dir_match[0])
    raster_files = sorted([os.path.join(ascii_dir, f) for f in os.listdir(ascii_dir) if f.endswith(".asc")])

    # Create output folder for this isotope
    isotope_outdir = os.path.join(PATH_OUTPUT, isotope)
    os.makedirs(isotope_outdir, exist_ok=True)

    # Loop over catchments
    for _, catch in tqdm.tqdm(catchments.iterrows(), total=len(catchments), desc=f"Catchments for {isotope}"):
        catch_id = catch["gauge_id"]
        catch_geom = gpd.GeoDataFrame([catch], crs=catchments.crs)

        values = []
        months = []

        for file_path in raster_files:
            # Extract date from filename
            month_match = re.search(r"_(\d{8})\.asc$", file_path)
            if not month_match:
                continue
            date_str = month_match.group(1)
            date = pd.to_datetime(date_str, format="%Y%m%d")
            months.append(date)

            mean_value = extract_mean_per_catchment(file_path, catch_geom)
            values.append(mean_value)

        # Save timeseries for this catchment
        df = pd.DataFrame({
            "date": months,
            f"{isotope}_monthlymean": values
        })
        df.to_csv(os.path.join(isotope_outdir, f"{catch_id}_isoscape.csv"),
                  index=False, sep=";", float_format="%.2f")


🧪 Processing isotope 18O...


Catchments for 18O:  10%|▉         | 11/115 [09:57<1:34:09, 54.32s/it]


KeyboardInterrupt: 

Eventually, we generate the yearly mean isoscapes:

In [19]:
for isotope in isotopes:
    print(f"\n📈 Generating yearly summary for {isotope}...")

    # Directory with catchment time series for this isotope
    iso_dir = os.path.join(PATH_OUTPUT, isotope)
    files = [f for f in os.listdir(iso_dir) if f.endswith("_isoscape.csv")]

    # Preallocate result DataFrame
    yearly_data = {}

    for file in files:
        catch_id = file.replace("_isoscape.csv", "")
        filepath = os.path.join(iso_dir, file)

        df = pd.read_csv(filepath, sep=";", parse_dates=["date"])
        df["year"] = df["date"].dt.year

        # Column is like "18O_monthlymean" or "2H_monthlymean"
        val_col = [col for col in df.columns if col.endswith("_monthlymean")][0]

        yearly_mean = df.groupby("year")[val_col].mean()
        yearly_data[catch_id] = yearly_mean

    # Combine into one DataFrame (rows = years, columns = catchments)
    yearly_df = pd.DataFrame(yearly_data)
    yearly_df.index.name = "year"

    # Save to CSV
    out_fp = os.path.join(PATH_OUTPUT, f"{isotope}_isoscape_yearly_summary.csv")
    yearly_df.to_csv(out_fp, sep=";", float_format="%.2f")

    print(f"Saved yearly summary to: {out_fp}")


📈 Generating yearly summary for 18O...
Saved yearly summary to: S:\CAMELS-CH\CAMELS-chem\results\catchment_aggregated_data\isoscapes\18O_isoscape_yearly_summary.csv

📈 Generating yearly summary for 2H...
Saved yearly summary to: S:\CAMELS-CH\CAMELS-chem\results\catchment_aggregated_data\isoscapes\2H_isoscape_yearly_summary.csv


Organize the data as time-series for the dataset (stored at the results folder)

In [ ]:
import os
import pandas as pd

folder_2h = '../../results/isoscapes/2H'
folder_18o = '../../results/isoscapes/18O'
output_folder = "../../results/Dataset/catchment_aggregated_data/rain_water_isotopes"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all files in the 2H folder
files_2h = sorted([f for f in os.listdir(folder_2h) if f.endswith('.csv')])

for filename in files_2h:
    path_2h = os.path.join(folder_2h, filename)
    path_18o = os.path.join(folder_18o, filename)

    # Read both files
    df_2h = pd.read_csv(path_2h, sep=';', parse_dates=['date'])
    df_18o = pd.read_csv(path_18o, sep=';', parse_dates=['date'])

    # Rename columns
    df_2h.rename(columns={'2H_monthlymean': 'delta_2h'}, inplace=True)
    df_18o.rename(columns={'18O_monthlymean': 'delta_18o'}, inplace=True)

    # Merge on date
    df_merged = pd.merge(df_2h, df_18o, on='date')

    # Set index
    df_merged.set_index('date', inplace=True)
    df_merged = df_merged.loc[:"2020", :]
    # Save the output
    output_path = os.path.join(output_folder, f"camels_ch_chem_rainisotopes_{filename[0:4]}.csv")
    df_merged.to_csv(output_path)